Christine Plumejeaud-Perreau, UMR MIGRINTER 7301

02 may 2022
projet ANR PORTIC

# Traitement des données de raw_flows pour regroupement par destination et analyse des produits et tonnages expédiés
# Analyse sur 1787 et France entière. 

Note 1787 : Oui, on a les données de Bayonne. 

In [56]:
import pandas as pd
import pandas.core.frame as pf

import json, urllib
import plotly as py
import pandas as pd
import numpy as np
import plotly.graph_objects as go


In [57]:

type(raw_flows) #list

print(raw_flows.shape) #(30722, 17)
print(raw_flows.dtypes)# Récupérer les raw_flows
#http://data.portic.fr/api/rawflows/?format=csv&date=1787&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo

#curl -o ./raw_flows_1789.csv "data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo"
raw_flows = pd.read_csv("./raw_flows_1787.csv", encoding='utf8', sep=',')


print(raw_flows.columns)

''' 'Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo' '''

(10081, 17)
Unnamed: 0                                 int64
departure_uhgs_id                         object
departure_fr                              object
destination_uhgs_id                       object
destination_fr                            object
destination_partner_balance_1789          object
destination_partner_balance_supp_1789     object
destination_substate_1789_fr              object
destination_state_1789_fr                 object
tonnage                                  float64
outdate_fixed                             object
commodity_purpose                         object
commodity_purpose2                        object
commodity_purpose3                        object
commodity_purpose4                        object
all_cargos                                object
nb_cargo                                 float64
dtype: object
Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance

" 'Unnamed: 0', 'departure_uhgs_id', 'departure_fr',\n       'destination_uhgs_id', 'destination_fr',\n       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',\n       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',\n       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',\n       'commodity_purpose4', 'all_cargos', 'nb_cargo' "

    'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nbproduits'

In [58]:
# Filtrer : oui, la pêche et le lest et ce qui part vers la France 
# Mais tagger les valeurs expédiées en GB particulièrement ? Oui. 

# dunkerque = raw_flows.query('departure_fr=="Dunkerque"')
# print(dunkerque.shape)
france = raw_flows.query('destination_partner_balance_supp_1789=="Etranger"') #Pas la France
france = france.query('destination_uhgs_id!="C0000009"') #Afrique
france = france.query('destination_uhgs_id!="A0146289"') #Islande
france = france.query('destination_uhgs_id!="B0000715"')  #Terre-Neuve
france = france.query('commodity_purpose!="Lège"')  #pas à vide (Lège)
france = france.query('commodity_purpose!="lest"')  #pas à vide (lest)
france = france.query('commodity_purpose2!="lest"')  #pas à vide (lest)
france = france.query('commodity_purpose!="pêche"')  #pas pour la pêche
print(france.shape) #(193, 16)


print(france.columns)

#dunkerque = dunkerque.query('destination_state_1789_fr!="Grande-Bretagne"') #Pas le smogglage vers l'angleterre

france = france.drop(['Unnamed: 0'], axis=1) #'all_cargos'
print(france.columns)

print(france.shape) #(7836, 16)

#Il arrive que des cargos soient na 

values = {"all_cargos": '', "nb_cargo": 0}
france = france.fillna(value=values)
#france = france.dropna(subset=['all_cargos'])
print(france.shape) #(7836, 16)
# GROS trou dans la rquette : 7836 - 4970 disparaissent car porduits non renseignés

france.to_csv (r'./flux_france_1787.csv', index = None)




(7836, 17)
Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')
Index(['departure_uhgs_id', 'departure_fr', 'destination_uhgs_id',
       'destination_fr', 'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')
(7836, 16)
(7836, 16)


## Premier apercu des destinations d'export de France vers l'étranger, AVEC GB, MAIS hors pêche, hors navires à vide. 

In [59]:

#Note, il y a une difference entre size et count, car il y a des navires avec tonnage inconnu
gdata = pf.DataFrame({'count' : france.groupby( [ "destination_partner_balance_1789"] ).size()}).reset_index()
print(gdata)

pf.DataFrame(france.groupby([ "destination_partner_balance_1789"])['tonnage'].agg(['count','sum'])).reset_index()


   destination_partner_balance_1789  count
0                        Angleterre   4244
1                          Danemark    429
2                           Espagne    654
3               Etats de l'Empereur    305
4                        Etats-Unis    189
5                          Hollande    643
6                          Portugal    175
7                            Prusse     98
8        Quatre villes hanséatiques    348
9                            Russie     68
10                            Suède    110


,destination_partner_balance_1789,count,sum
0,Angleterre,4088,153070.0
1,Danemark,427,42054.0
2,Espagne,615,46889.0
3,Etats de l'Empereur,304,28720.0
4,Etats-Unis,188,32969.0
5,Hollande,641,74228.0
6,Portugal,173,23302.0
7,Prusse,98,18268.0
8,Quatre villes hanséatiques,348,65575.0
9,Russie,68,14063.0


In [60]:
import csv
import json
import ast

def unstack_cargos_in_pointcalls(df):
  by_pointcalls = []

  for _, row in df.iterrows():
    if row['all_cargos'] != '' and row['all_cargos'] != 'nan':
      truc = row['all_cargos']
      
      #truc = truc.replace("'", '"').replace("None", 'null')
      #.replace("bœ\x9c\x9cuf", "boeuf")
      #print(json.dumps(truc))
      #truc = json.dumps(truc)
      #print(truc)
      #cargos = json.loads(truc)
      cargos = ast.literal_eval(truc) #MAGIE
      #print(type(cargos))
      for cargo in cargos:
        if  cargo['commodity_standardized_fr'] != 'Lest' : 
          if  cargo['commodity_standardized_fr'] != None : 
            by_pointcalls.append({
              "cargo": cargo['commodity_purpose'],
              "cargo_standardized_fr": cargo['commodity_standardized_fr'],
              **row
            })
          else :
            by_pointcalls.append({
              "cargo": cargo['commodity_purpose'],
              "cargo_standardized_fr": cargo['commodity_purpose'].lower(),
              **row
            })
    else:
      by_pointcalls.append({
        "cargo": "",
        "cargo_standardized_fr": "",
        **row
      })
  return by_pointcalls

#

In [61]:
#USUED CODE FOR THE MOMENT


from dunkerquesprint import build_toflit18_classif_multimap, get_online_csv

def get_toflit18_classification(product_classification="product_simplification"):
    # créer un dict dont chaque clé sera une des classifications à parcourir,
  # et chaque valeur un dict dont les clés sont les formes parentes et les valeurs les formes enfant
  classification_path, classif_multi_dict = build_toflit18_classif_multimap(product_classification)
  prev_classif = 'product_source'
  for classif in classification_path:
    classif_multi_dict[classif] = {}
    # on créée un dict alternatif avec les valeurs en lowercase pour parer à d'éventuels problèmes liés à la casse
    classif_multi_dict[classif + '_lower'] = {}
    # récupérer le csv à jour de la classification sur le repo toflit18_data
    toflit18_csv_url = 'https://raw.githubusercontent.com/medialab/toflit18_data/master/base/classification_' + classif + '.csv'
    # télécharger le csv depuis toflit18_data
    classif_data = get_online_csv(toflit18_csv_url)
    prev_key = prev_classif.split('product_')[1]
    current_key = classif.split('product_')[1]
    for row in classif_data:
      # nom de la classification "parent" : e.g. "orthographic"
      parent_value = row[prev_key]
      # nom de la classification "enfant" : e.g. "simplification"
      child_value = row[current_key]
      classif_multi_dict[classif][parent_value] = child_value
      # gérer les problèmes de casse en stockant dans le dict alternatif la valeur originale et la valeur en lower
      classif_multi_dict[classif + '_lower'][parent_value.lower()] = {
        "original": child_value,
        "lower" : child_value.lower()
      }
    prev_classif = classif

  return prev_classif


In [62]:
# Traduire les produits de navigo en produits de toflit

from dunkerquesprint import build_toflit18_classif_multimap
product_classification="product_simplification"
#product_classification="product_RE_aggregate"

#product_classification="product_reexportations"
classification_path, classif_multi_dict = build_toflit18_classif_multimap(product_classification)

NAN_TOFLIT = "Unknown for TOFLIT"

def translate_as_toflit_product(source_name):
  # créer un dict dont chaque clé sera une des classifications à parcourir,
  # et chaque valeur un dict dont les clés sont les formes parentes et les valeurs les formes enfant
  
  # la valeur PORTIC cargo_standardized_fr correspond au niveau "source" des classifications TOFLIT18
  #source_name = pointcall["cargo_standardized_fr"]
  translated_name = None
  if source_name is not None:
    # on stocke dans une valeur courante la traduction TOFLIT18 (qui va par exemple correspondre successivement à source -> orthographic -> simplification)
    translated_name = source_name
    # parcourir les classifs pour trouver la bonne valeur
    for classif in classification_path:
      # si la valeur est dans le dict de la classif toflit18 courante on le traduit
      if translated_name is not None and translated_name in classif_multi_dict[classif]:
        translated_name = classif_multi_dict[classif][translated_name]
      # si la valeur matche en lowercase on la traduit aussi
      elif translated_name is not None and translated_name.lower() in classif_multi_dict[classif + '_lower']:
        translated_name = classif_multi_dict[classif + '_lower'][translated_name.lower()]["original"]
      # si pas de valeur trouvée => alignement impossible, besoin de màj côté toflit18 pour intégrer cette forme
      else:
        translated_name = NAN_TOFLIT
  #pointcall["commodity_as_toflit"] = translated_name
  return translated_name

In [63]:
by_pointcalls =  unstack_cargos_in_pointcalls(france)
export_france_1787 = pd.DataFrame(by_pointcalls, columns=['cargo', 'cargo_standardized_fr']+france.columns.tolist())

#print(export_france_1787.columns)

export_france_1787['commodity_as_toflit'] = export_france_1787['cargo_standardized_fr'].apply(translate_as_toflit_product)

export_france_1787 = export_france_1787.drop(['commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos'], axis=1)

print(export_france_1787.columns)
print(export_france_1787.shape) #(9337, 14)
export_france_1787.to_csv (r'./export_france_1787.csv', index = None)


Index(['cargo', 'cargo_standardized_fr', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'nb_cargo',
       'commodity_as_toflit'],
      dtype='object')
(9337, 14)


In [64]:
import pandas as pd
import pandas.core.frame as pf

# Lire le fichier 
#export_france_1787 = pd.read_csv("./export_france_1787.csv")
export_france_1787['tonnage_per_product'] = export_france_1787['tonnage'] / export_france_1787['nb_cargo']

gdata = pf.DataFrame({'count' : export_france_1787.groupby( [ "destination_partner_balance_1789","commodity_as_toflit"] ).size()}).reset_index()
#print(gdata)

pd.DataFrame(export_france_1787.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage'].agg(['sum','count', 'mean'])).reset_index()

exports = pd.DataFrame(export_france_1787.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()
exports.set_axis(['destination_partner_balance_1789', 'commodity_as_toflit', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'], axis=1, inplace=True)
exports.to_csv (r'./export_france_1787_par_pays_par_produits.csv', index = None)



C:\Users\cplume01\AppData\Local\Temp\ipykernel_15796\1177664358.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  exports = pd.DataFrame(export_france_1787.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()


# Pour 1789, on s'intéresse à la même chose pour Dunkerque, La Rochelle et Marseille
# ie la navigation pour l'étranger sans navires à vide, sans pêche

In [65]:

#http://data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo

#curl -o ./raw_flows_1789.csv "data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo"
raw_flows = pd.read_csv("./raw_flows_1789.csv", encoding='utf8', sep=',')


print(raw_flows.columns)

#Filtrer : oui, la pêche et le lest et ce qui part vers la France 
# Mais tagger les valeurs expédiées en GB particulièrement ? Oui. 

dunkerque = raw_flows.query('departure_fr=="Dunkerque"')
# print(dunkerque.shape)
dunkerque = dunkerque.query('destination_partner_balance_supp_1789=="Etranger"') #Pas la France
dunkerque = dunkerque.query('destination_uhgs_id!="C0000009"') #Afrique
dunkerque = dunkerque.query('destination_uhgs_id!="A0146289"') #Islande
dunkerque = dunkerque.query('destination_uhgs_id!="B0000715"')  #Terre-Neuve
dunkerque = dunkerque.query('commodity_purpose!="Lège"')  #pas à vide (Lège)
dunkerque = dunkerque.query('commodity_purpose!="lest"')  #pas à vide (lest)
dunkerque = dunkerque.query('commodity_purpose2!="lest"')  #pas à vide (lest)
dunkerque = dunkerque.query('commodity_purpose!="pêche"')  #pas pour la pêche
print(dunkerque.shape) #(1239, 17)


#print(dunkerque.columns)

#dunkerque = dunkerque.query('destination_state_1789_fr!="Grande-Bretagne"') #Pas le smogglage vers l'angleterre

dunkerque = dunkerque.drop(['Unnamed: 0'], axis=1) #'all_cargos'
print(dunkerque.shape) #(1239, 16)

#Il arrive que des cargos soient na 

produits_connus = dunkerque.dropna(subset=['all_cargos'])
#print(produits_connus.shape) #(1239, 16)

values = {"all_cargos": '', "nb_cargo": 0}
dunkerque = dunkerque.fillna(value=values)


## Maintenant on duplique les lignes : une par produit, et on aligne avec les produits de Toflit'18

by_pointcalls =  unstack_cargos_in_pointcalls(dunkerque)
export_dunkerque_1789 = pd.DataFrame(by_pointcalls, columns=['cargo', 'cargo_standardized_fr']+dunkerque.columns.tolist())

#print(export_dunkerque_1789.columns)

export_dunkerque_1789['commodity_as_toflit'] = export_dunkerque_1789['cargo_standardized_fr'].apply(translate_as_toflit_product)
export_dunkerque_1789 = export_dunkerque_1789.drop(['commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos'], axis=1)

print(export_dunkerque_1789.columns)
print(export_dunkerque_1789.shape) #(2192, 14)

export_dunkerque_1789.to_csv (r'./suite_flux_dunkerque_1789.csv', index = None)



Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')
(1239, 17)
(1239, 16)
Index(['cargo', 'cargo_standardized_fr', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'nb_cargo',
       'commodity_as_toflit'],
      dtype='object')
(2192, 14)


### Regroupement par produit et par partenaire toflit des exports de Dunkerque en 1789


In [66]:
import pandas as pd
import pandas.core.frame as pf

# Lire le fichier 
#export_dunkerque_1789 = pd.read_csv("./suite_flux_dunkerque_1789.csv")
export_dunkerque_1789['tonnage_per_product'] = export_dunkerque_1789['tonnage'] / export_dunkerque_1789['nb_cargo']

gdata = pf.DataFrame({'count' : export_dunkerque_1789.groupby( [ "destination_partner_balance_1789","commodity_as_toflit"] ).size()}).reset_index()
print(gdata)

pd.DataFrame(export_dunkerque_1789.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage'].agg(['sum','count', 'mean'])).reset_index()

exports = pd.DataFrame(export_dunkerque_1789.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()
exports.set_axis(['destination_partner_balance_1789', 'commodity_as_toflit', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'], axis=1, inplace=True)
exports.to_csv (r'./suite_export_dunkerque_1789_par_pays_par_produits.csv', index = None)


   destination_partner_balance_1789 commodity_as_toflit  count
0                        Angleterre  Unknown for TOFLIT     11
1                        Angleterre              [vide]      1
2                        Angleterre             batiste      1
3                        Angleterre    bois de Campêche      1
4                        Angleterre             briques      1
..                              ...                 ...    ...
86       Quatre villes hanséatiques        marchandises      1
87       Quatre villes hanséatiques               tabac      3
88       Quatre villes hanséatiques                 vin      1
89                            Suède  Unknown for TOFLIT      1
90                            Suède            genièvre      1

[91 rows x 3 columns]


C:\Users\cplume01\AppData\Local\Temp\ipykernel_15796\2232765458.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  exports = pd.DataFrame(export_dunkerque_1789.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()


### pour La Rochelle, en 1789

Attention, à la Rochelle, on ne connait jamais le produit car non renseigné. 



In [67]:
# Prendre les données
##departure_ferme_bureau, departure_action 
#departure_ferme_bureau='La Rochelle'
#Equivalent à ces ports
#    Esnandes
#    La Rochelle
#    mer des Pertuis

#http://data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo

#curl -o ./raw_flows_1789.csv "data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo"
raw_flows = pd.read_csv("./raw_flows_1789.csv", encoding='utf8', sep=',')


print(raw_flows.columns)

#Filtrer : oui, la pêche et le lest et ce qui part vers la France 
# Mais tagger les valeurs expédiées en GB particulièrement ? Oui. 

lRochelle = raw_flows.query('departure_fr=="La Rochelle" or departure_fr=="Esnandes" or departure_fr=="mer des Pertuis"')
# print(lRochelle.shape)
lRochelle = lRochelle.query('destination_partner_balance_supp_1789=="Etranger"') #Pas la France
lRochelle = lRochelle.query('destination_uhgs_id!="C0000009"') #Afrique
lRochelle = lRochelle.query('destination_uhgs_id!="A0146289"') #Islande
lRochelle = lRochelle.query('destination_uhgs_id!="B0000715"')  #Terre-Neuve
lRochelle = lRochelle.query('commodity_purpose!="Lège"')  #pas à vide (Lège)
lRochelle = lRochelle.query('commodity_purpose!="lest"')  #pas à vide (lest)
lRochelle = lRochelle.query('commodity_purpose2!="lest"')  #pas à vide (lest)
lRochelle = lRochelle.query('commodity_purpose!="pêche"')  #pas pour la pêche
print(lRochelle.shape) #(35, 17)


#print(lRochelle.columns)

#dunkerque = dunkerque.query('destination_state_1789_fr!="Grande-Bretagne"') #Pas le smogglage vers l'angleterre

lRochelle = lRochelle.drop(['Unnamed: 0'], axis=1) #'all_cargos'
print(lRochelle.shape) #(35, 16)

#Il arrive que des cargos soient na 

produits_connus = lRochelle.dropna(subset=['all_cargos'])
print("Nombre de départs avec produits connus")
print(produits_connus.shape) #(35, 16)

values = {"all_cargos": '', "nb_cargo": 0}
lRochelle = lRochelle.fillna(value=values)


## Maintenant on duplique les lignes : une par produit, et on aligne avec les produits de Toflit'18

by_pointcalls =  unstack_cargos_in_pointcalls(lRochelle)
export_lRochelle_1789 = pd.DataFrame(by_pointcalls, columns=['cargo', 'cargo_standardized_fr']+lRochelle.columns.tolist())

#print(export_lRochelle_1789.columns)

export_lRochelle_1789['commodity_as_toflit'] = export_lRochelle_1789['cargo_standardized_fr'].apply(translate_as_toflit_product)
export_lRochelle_1789 = export_lRochelle_1789.drop(['commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos'], axis=1)

print(export_lRochelle_1789.columns)
print(export_lRochelle_1789.shape) #(35, 14)

export_lRochelle_1789.to_csv (r'./suite_flux_lRochelle_1789.csv', index = None)

Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')
(35, 17)
(35, 16)
Nombre de départs avec produits connus
(0, 16)
Index(['cargo', 'cargo_standardized_fr', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'nb_cargo',
       'commodity_as_toflit'],
      dtype='object')
(35, 14)


In [68]:
## Agréger La Rochelle 1789 par partenaire

import pandas as pd
import pandas.core.frame as pf

# Lire le fichier 
#export_lRochelle_1789 = pd.read_csv("./suite_flux_lRochelle_1789.csv")

# Dans ce cas, avec des produits jamais renseignés, on a nb_cargo = 0, et tonnage_per_product n'a pas de sens
#export_lRochelle_1789['tonnage_per_product'] = export_lRochelle_1789['tonnage'] / export_lRochelle_1789['nb_cargo']
#Diviser par 0 donne Infinity, donc on ne garde pas cette variable

exports = pd.DataFrame(export_lRochelle_1789.groupby([ "destination_partner_balance_1789"])['tonnage'].agg(['count','sum', 'mean'])).reset_index()
print(exports)

#exports = pd.DataFrame(export_lRochelle_1789.groupby([ "destination_partner_balance_1789"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()
exports.set_axis(['destination_partner_balance_1789', 'nb_ships', 'sum_tonnage', 'tonnage moyen'], axis=1, inplace=True)
exports.to_csv (r'./suite_export_laRochelle_1789_par_pays_par_produits.csv', index = None)


  destination_partner_balance_1789  count     sum        mean
0                       Angleterre      6  1048.0  174.666667
1                         Danemark      1   150.0  150.000000
2              Etats de l'Empereur      8   512.0   64.000000
3                       Etats-Unis      1   160.0  160.000000
4                         Hollande      5   550.0  110.000000
5                           Prusse      3   370.0  123.333333
6       Quatre villes hanséatiques     10  1050.0  105.000000


## La Rochelle _ 1787 
Est-ce que là aussi les produits sont absents ? OUI.

In [69]:
# Prendre les données
##departure_ferme_bureau, departure_action 
#departure_ferme_bureau='La Rochelle'
#Equivalent à ces ports
#    Esnandes
#    La Rochelle
#    mer des Pertuis

#http://data.portic.fr/api/rawflows/?format=csv&date=1787&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo

#curl -o ./raw_flows_1789.csv "data.portic.fr/api/rawflows/?format=csv&date=1787&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo"
raw_flows = pd.read_csv("./raw_flows_1787.csv", encoding='utf8', sep=',')


print(raw_flows.columns)

#Filtrer : oui, la pêche et le lest et ce qui part vers la France 
# Mais tagger les valeurs expédiées en GB particulièrement ? Oui. 

lRochelle = raw_flows.query('departure_fr=="La Rochelle" or departure_fr=="Esnandes" or departure_fr=="mer des Pertuis"')
# print(lRochelle.shape)
lRochelle = lRochelle.query('destination_partner_balance_supp_1789=="Etranger"') #Pas la France
lRochelle = lRochelle.query('destination_uhgs_id!="C0000009"') #Afrique
lRochelle = lRochelle.query('destination_uhgs_id!="A0146289"') #Islande
lRochelle = lRochelle.query('destination_uhgs_id!="B0000715"')  #Terre-Neuve
lRochelle = lRochelle.query('commodity_purpose!="Lège"')  #pas à vide (Lège)
lRochelle = lRochelle.query('commodity_purpose!="lest"')  #pas à vide (lest)
lRochelle = lRochelle.query('commodity_purpose2!="lest"')  #pas à vide (lest)
lRochelle = lRochelle.query('commodity_purpose!="pêche"')  #pas pour la pêche
print(lRochelle.shape) #(37, 17)


#print(lRochelle.columns)

#dunkerque = dunkerque.query('destination_state_1789_fr!="Grande-Bretagne"') #Pas le smogglage vers l'angleterre

lRochelle = lRochelle.drop(['Unnamed: 0'], axis=1) #'all_cargos'
print(lRochelle.shape) #(37, 16)

#Il arrive que des cargos soient na 

produits_connus = lRochelle.dropna(subset=['all_cargos'])
print("Nombre de départs avec produits connus")
print(produits_connus.shape) #(37, 16)

values = {"all_cargos": '', "nb_cargo": 0}
lRochelle = lRochelle.fillna(value=values)


## Maintenant on duplique les lignes : une par produit, et on aligne avec les produits de Toflit'18

by_pointcalls =  unstack_cargos_in_pointcalls(lRochelle)
export_lRochelle_1787 = pd.DataFrame(by_pointcalls, columns=['cargo', 'cargo_standardized_fr']+lRochelle.columns.tolist())

#print(export_lRochelle_1787.columns)

export_lRochelle_1787['commodity_as_toflit'] = export_lRochelle_1787['cargo_standardized_fr'].apply(translate_as_toflit_product)
export_lRochelle_1787 = export_lRochelle_1787.drop(['commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos'], axis=1)

print(export_lRochelle_1787.columns)
print(export_lRochelle_1787.shape) #(37, 14)

export_lRochelle_1787.to_csv (r'./suite_flux_lRochelle_1787.csv', index = None)

Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')
(37, 17)
(37, 16)
Nombre de départs avec produits connus
(0, 16)
Index(['cargo', 'cargo_standardized_fr', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'nb_cargo',
       'commodity_as_toflit'],
      dtype='object')
(37, 14)


In [70]:
# agréger par produit, partenaire toflit (qui peut le plus peu le moins)

## Agréger La Rochelle 1789 par partenaire
# Dans ce cas, avec des produits jamais renseignés, on a nb_cargo = 0, et tonnage_per_product n'a pas de sens

import pandas as pd
import pandas.core.frame as pf

# Lire le fichier 
#export_lRochelle_1787 = pd.read_csv("./suite_flux_lRochelle_1787.csv")

exports = pd.DataFrame(export_lRochelle_1787.groupby([ "destination_partner_balance_1789"])['tonnage'].agg(['count','sum', 'mean'])).reset_index()
print(exports)

exports.set_axis(['destination_partner_balance_1789', 'nb_ships', 'sum_tonnage', 'tonnage moyen'], axis=1, inplace=True)
exports.to_csv (r'./suite_export_laRochelle_1787_par_pays_par_produits.csv', index = None)




  destination_partner_balance_1789  count     sum        mean
0                       Angleterre      4   770.0  192.500000
1                         Danemark      3   360.0  120.000000
2                          Espagne      3   230.0   76.666667
3              Etats de l'Empereur      9   996.0  110.666667
4                         Hollande      4   490.0  122.500000
5                         Portugal      2   214.0  107.000000
6       Quatre villes hanséatiques     12  1324.0  110.333333


## Marseille, 1789
Il y aura plusieurs sources. Et le petit cabotage sort parfois en doublon du G5 (net_route_marker = Q)
Expéditions "coloniales" Marseille (1789)
G5
Registre du petit cabotage (1786-1787)
Santé Marseille

Une petit préanalayse en SQL m'a montré que la requête ne me sort pas de doublon (departure_ferme_bureau='Marseille' and destination_partner_balance_supp_1789='Etranger') parce que je controle le net_route_marker des raw_flows à la destination.
Bref, seule l'analyse 1789 ramène une cinquantaine de lignes. 

In [71]:
# Prendre les données
#departure_ferme_bureau='Marseille'
#Equivalent à ces ports
#    Frioul
#    Marseille

#http://data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo

#curl -o ./raw_flows_1789.csv "data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_1789,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo"
raw_flows = pd.read_csv("./raw_flows_1789.csv", encoding='utf8', sep=',')


print(raw_flows.columns)

#Filtrer : oui, la pêche et le lest et ce qui part vers la France 
# Mais tagger les valeurs expédiées en GB particulièrement ? Oui. 

Marseille_1789 = raw_flows.query('departure_fr=="Marseille" or departure_fr=="Frioul"')
# print(lRochelle.shape)
Marseille_1789 = Marseille_1789.query('destination_partner_balance_supp_1789=="Etranger"') #Pas la France
Marseille_1789 = Marseille_1789.query('destination_uhgs_id!="C0000009"') #Afrique
Marseille_1789 = Marseille_1789.query('destination_uhgs_id!="A0146289"') #Islande
Marseille_1789 = Marseille_1789.query('destination_uhgs_id!="B0000715"')  #Terre-Neuve
Marseille_1789 = Marseille_1789.query('commodity_purpose!="Lège"')  #pas à vide (Lège)
Marseille_1789 = Marseille_1789.query('commodity_purpose!="lest"')  #pas à vide (lest)
Marseille_1789 = Marseille_1789.query('commodity_purpose2!="lest"')  #pas à vide (lest)
Marseille_1789 = Marseille_1789.query('commodity_purpose!="pêche"')  #pas pour la pêche
print(Marseille_1789.shape) #(42, 17)


#dunkerque = dunkerque.query('destination_state_1789_fr!="Grande-Bretagne"') #Pas le smogglage vers l'angleterre

Marseille_1789 = Marseille_1789.drop(['Unnamed: 0'], axis=1) #'all_cargos'

#Il arrive que des cargos soient na 

produits_connus = Marseille_1789.dropna(subset=['all_cargos'])
print("Nombre de départs avec produits connus")
print(produits_connus.shape) #(42, 16)

values = {"all_cargos": '', "nb_cargo": 0}
Marseille_1789 = Marseille_1789.fillna(value=values)


## Maintenant on duplique les lignes : une par produit, et on aligne avec les produits de Toflit'18

by_pointcalls =  unstack_cargos_in_pointcalls(Marseille_1789)
export_Marseille_1789 = pd.DataFrame(by_pointcalls, columns=['cargo', 'cargo_standardized_fr']+Marseille_1789.columns.tolist())


export_Marseille_1789['commodity_as_toflit'] = export_Marseille_1789['cargo_standardized_fr'].apply(translate_as_toflit_product)
export_Marseille_1789 = export_Marseille_1789.drop(['commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos'], axis=1)

print(export_Marseille_1789.columns)
print(export_Marseille_1789.shape) #(238, 14)

export_Marseille_1789.to_csv (r'./suite_flux_Marseille_1789.csv', index = None)

Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')
(42, 17)
Nombre de départs avec produits connus
(42, 16)
Index(['cargo', 'cargo_standardized_fr', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_1789',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'nb_cargo',
       'commodity_as_toflit'],
      dtype='object')
(238, 14)


In [72]:
## Agréger à Marseille 1789


import pandas as pd
import pandas.core.frame as pf

# Lire le fichier 
#export_Marseille_1789 = pd.read_csv("./suite_flux_dunkerque_1789.csv")
export_Marseille_1789['tonnage_per_product'] = export_Marseille_1789['tonnage'] / export_Marseille_1789['nb_cargo']

gdata = pf.DataFrame({'count' : export_Marseille_1789.groupby( [ "destination_partner_balance_1789","commodity_as_toflit"] ).size()}).reset_index()
print(gdata)

pd.DataFrame(export_Marseille_1789.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage'].agg(['sum','count', 'mean'])).reset_index()

exports = pd.DataFrame(export_Marseille_1789.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()
exports.set_axis(['destination_partner_balance_1789', 'commodity_as_toflit', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'], axis=1, inplace=True)
exports.to_csv (r'./suite_export_marseille_1789_par_pays_par_produits.csv', index = None)




   destination_partner_balance_1789     commodity_as_toflit  count
0                           Espagne      Unknown for TOFLIT      1
1                           Espagne                    bois      1
2                           Espagne  bois pour construction      2
3                           Espagne                 cercles      1
4                           Espagne             chair salée      1
..                              ...                     ...    ...
59                         Portugal                   savon      2
60                         Portugal                souliers      1
61                         Portugal           toile à voile      1
62                         Portugal                verrerie      2
63                         Portugal                     vin      3

[64 rows x 3 columns]


C:\Users\cplume01\AppData\Local\Temp\ipykernel_15796\4131446049.py:16: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  exports = pd.DataFrame(export_Marseille_1789.groupby([ "destination_partner_balance_1789","commodity_as_toflit"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()



Christine Plumejeaud-Perreau, UMR MIGRINTER 7301
03 mai 2022
projet ANR PORTIC
